In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('stock_news.csv')
df.head()
print(df['Dates'][0]==df['Dates'][1])
al=[]
news=[]
for i in range(len(df)-1):
    
    if(df['Dates'][i]!=df['Dates'][i+1]) :
        news.append(df['HeadLine'][i])
        al.append(news)
        
        #print(news)
        news=[]
    else:
        news.append(df['HeadLine'][i])
#print(al)
print(len(al))
#add random future date to end(no include last date data.)
df.drop(df.tail(1).index,inplace=True)
print(len(df.Dates.unique()))
print(df.tail(1))
print(al[-1])
print('')
print(df.head(1))
print(al[0])

True
2552
2552
                                                HeadLine      day     Dates
19938  Sensex rises 200 pts; TCS, SBI, ICICI Bank, In...  Tuesday  8/7/2012
['RBS sees Nifty hitting 5,700 mark by December', 'Sensex ends 1% higher two-day in a row on reform hopes', 'Expect Nifty to test 5350 soon', 'Sensex rises 200 pts; TCS, SBI, ICICI Bank, Infosys gain 2%']

                                            HeadLine     day      Dates
0  Mutual funds make fresh buy in these 16 stocks...  Friday  4/16/2021
['Mutual funds make fresh buy in these 16 stocks, exit 8 in March', 'Where are the ultra-rich investing? Over 20 stocks from top 5 PMS funds outperformed Nifty in March', 'COVID takes toll, over 50% Nifty50 stocks are down 10-20% from highs', 'Trade Spotlight: What should investors do with Balrampur Chini, Wipro & HDFC Bank?', 'Hot Stocks | IPCA Laboratories, HDFC Life, Marico can give up to 33% return in short term', 'Market LIVE Updates: Sensex, Nifty trade higher; mid, smallc

In [4]:
import os
import copy
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertConfig
from bertModel import BertClassification, dense_opt
from datasets import text_dataset, financialPhraseBankDataset
import argparse
from sklearn.metrics import f1_score


ModuleNotFoundError: No module named 'bertModel'

In [2]:
dates = df.Dates.unique()

In [3]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
sid.polarity_scores(al[0][0])

{'neg': 0.0, 'neu': 0.827, 'pos': 0.173, 'compound': 0.3182}

In [5]:
print(al[0])

['Mutual funds make fresh buy in these 16 stocks, exit 8 in March', 'Where are the ultra-rich investing? Over 20 stocks from top 5 PMS funds outperformed Nifty in March', 'COVID takes toll, over 50% Nifty50 stocks are down 10-20% from highs', 'Trade Spotlight: What should investors do with Balrampur Chini, Wipro & HDFC Bank?', 'Hot Stocks | IPCA Laboratories, HDFC Life, Marico can give up to 33% return in short term', 'Market LIVE Updates: Sensex, Nifty trade higher; mid, smallcaps outperform']


In [6]:
scores=[]
for i in range(len(al)):
    score=0
    for j in range(len(al[i])):
        score+=sid.polarity_scores(al[i][j]).get('compound')
    avg_score = score/len(al[i])
    scores.append(avg_score)
print(len(scores))

2552


In [167]:
#creating the DF
rows = []

for i in range(len(al)):
     rows.append([dates[i], scores[i]])
print(rows[0:5])

df_new = pd.DataFrame(rows, columns=["Dates", "scores"])

#print(df_new)
df_new.to_csv('stock_scores_9.csv')

[['4/16/2021', 0.2104], ['4/15/2021', 0.17027499999999998], ['4/14/2021', 0.25899999999999995], ['4/13/2021', 0.2482333333333333], ['4/12/2021', 0.134625]]


In [168]:
#adding the scores to the orignal data set
df_main = pd.read_csv('stock_data_9.csv')
#df_main = df_main1.dropna()
df_main.head()


,Date,Open,High,Low,Close,Adj Close,Volume
0,2012-08-07,5295.399902,5350.100098,5281.649902,5336.700195,5336.700195,0.0
1,2012-08-08,5345.250000,5377.600098,5331.049805,5338.000000,5338.000000,0.0
2,2012-08-09,5348.299805,5368.200195,5312.100098,5322.950195,5322.950195,0.0
3,2012-08-10,5308.200195,5330.100098,5294.100098,5320.399902,5320.399902,0.0
4,2012-08-13,5316.350098,5352.450195,5309.049805,5347.899902,5347.899902,0.0


In [94]:
#Formating main data df to date match date column of scroes dates
#DONT RUN
df_main['Date'] = pd.to_datetime(df_main.Date, format='%Y-%m-%d')
f_df = df_main["Date"].dt.strftime("%m/%d/%Y")
f_dates = []
for i in range(len(f_df)):
    f_dates.append(f_df[i])
print(len(f_dates))
print(len(df_main))
df_main.Date = f_dates
df_main.head()

2142
2142


,Date,Open,High,Low,Close,Adj Close,Volume
0,08/07/2012,5295.399902,5350.100098,5281.649902,5336.700195,5336.700195,0.0
1,08/08/2012,5345.250000,5377.600098,5331.049805,5338.000000,5338.000000,0.0
2,08/09/2012,5348.299805,5368.200195,5312.100098,5322.950195,5322.950195,0.0
3,08/10/2012,5308.200195,5330.100098,5294.100098,5320.399902,5320.399902,0.0
4,08/13/2012,5316.350098,5352.450195,5309.049805,5347.899902,5347.899902,0.0


In [169]:
#Formating df_new data df to date match date column of scroes dates
df_new['Dates'] = pd.to_datetime(df_new.Dates, format="%m/%d/%Y")
f_df = df_new["Dates"].dt.strftime("%Y-%m-%d")
f_dates = []
for i in range(len(f_df)):
    f_dates.append(f_df[i])
print(len(f_dates))
print(len(df_main))
df_new.Dates = f_dates
df_new.head()

2552
2142


,Dates,scores
0,2021-04-16,0.210400
1,2021-04-15,0.170275
2,2021-04-14,0.259000
3,2021-04-13,0.248233
4,2021-04-12,0.134625


In [170]:
#Reducing scores csv to correlate to data.(number of entries)
df_dates = []
for i in range(len(df_main)):
    df_dates.append(df_main.Date[i])
ukdates = []
for i in range(len(df_new.Dates)) :
    if(df_new.Dates[i] not in df_dates):
        ukdates.append(df_new.Dates[i])
print(len(ukdates))

528


In [171]:
#Reducing scores csv to correlate to data.(number of entries)
mis_dates = []
for i in range(len(df_new)):
    mis_dates.append(df_new.Dates[i])
misdates = []
for i in range(len(df_main.Date)) :
    if(df_main.Date[i] not in mis_dates):
        misdates.append(df_main.Date[i])
print(len(misdates))

118


In [172]:
#Total missing dates
for i in misdates:
    if(i not in ukdates):
        ukdates.append(i)
print(len(ukdates))
print(ukdates[0])

646
2021-04-14


In [173]:
#labels of main set to delete
newlabels = ukdates
print(len(newlabels))
for i in range(len(df_new)):
    if(df_new.Dates[i] in newlabels):
        newlabels.remove(df_new.Dates[i])
print(len(newlabels))
    

646
118


In [174]:
#labels of df_new set to delete
mainlabels = ukdates
print(len(mainlabels))
for i in range(len(df_main)):
    if(df_main.Date[i] in mainlabels):
        mainlabels.remove(df_main.Date[i])
print(len(mainlabels))

118
0


In [175]:
#df_new.set_index("Dates", inplace = True)
df_main.set_index("Date", inplace = True)

In [179]:
#removing unknown dates from both df_new and df_main
#df_new.set_index("Dates", inplace = True)

#df_new = df_new.drop(labels = mainlabels , axis= 0)
df_main = df_main.drop(labels = misdates , axis= 0)
len(df_main)
#df_main.head

2024

In [180]:
#df_new_final = df_new
df_main_final = df_main

In [192]:
#df_new_final.reset_index(inplace = True)
reversed_df = df_new_final.iloc[::-1]
#df1.iloc[::-1]
reversed_df.head

<bound method NDFrame.head of       level_0  index       Dates    scores
2023     2023   2023  2012-08-07  0.438000
2022     2022   2022  2012-08-08  0.182200
2021     2021   2021  2012-08-09 -0.009333
2020     2020   2020  2012-08-10  0.062186
2019     2019   2019  2012-08-13  0.277214
...       ...    ...         ...       ...
4           4      4  2021-04-09  0.280429
3           3      3  2021-04-12  0.134625
2           2      2  2021-04-13  0.248233
1           1      1  2021-04-15  0.170275
0           0      0  2021-04-16  0.210400

[2024 rows x 4 columns]>

In [191]:
df_main_final.head
df_main_final.to_csv('main.csv')

In [195]:
reversed_df.to_csv('scores_dated.csv')

In [202]:
df_main_final['scores'] = scores

In [204]:
df_main_final.to_csv('final_with_scores.csv')

2023    0.438000
2022    0.182200
2021   -0.009333
2020    0.062186
2019    0.277214
          ...   
4       0.280429
3       0.134625
2       0.248233
1       0.170275
0       0.210400
Name: scores, Length: 2024, dtype: float64

In [201]:
scores = []
for i in range(len(reversed_df)-1,-1,-1):
    scores.append(reversed_df.scores[i])
print(scores)

[0.43799999999999994, 0.1822, -0.009333333333333341, 0.062185714285714275, 0.27721428571428575, 0.3318375, 0.3905714285714286, 0.3222, 0.3662375, 0.19610999999999995, 0.26722999999999997, 0.1666090909090909, 0.08917272727272728, 0.17862499999999998, 0.19279000000000002, -0.04346, 0.06556999999999999, 0.23659999999999998, 0.09790909090909089, 0.07728181818181817, 0.23983000000000004, 0.24918, 0.66076, 0.17633749999999998, 0.05145555555555555, 0.26765999999999995, 0.23261111111111113, 0.35423, 0.29891666666666666, 0.1449333333333333, -0.16816, 0.2771181818181818, 0.19668999999999998, 0.31734, 0.09443636363636361, 0.39518, 0.18838, 0.32794545454545454, 0.1254222222222222, 0.355, 0.21977500000000003, 0.09230999999999998, 0.33899, -0.024788888888888878, 0.3433777777777778, -0.0099, 0.03731111111111111, 0.1943888888888889, 0.23674, 0.18579166666666666, 0.16168999999999997, 0.008487500000000009, 0.22359090909090906, 0.37726250000000006, 0.26756250000000004, 0.29889999999999994, 0.335588888888

In [3]:
import numpy as np
import pandas as pd
df1 = pd.read_csv('9year_1-2week_windowsize.csv')
df1.head(7)

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,scores,windowSize_2,windowSize_3,week_1,week_2
0,0,2012-08-07,5295.399902,5350.100098,5281.649902,5336.700195,5336.700195,0.0,0.438000,0.438000,0.438000,0.438000,0.438000
1,1,2012-08-08,5345.250000,5377.600098,5331.049805,5338.000000,5338.000000,0.0,0.182200,0.310100,0.182200,0.182200,0.182200
2,2,2012-08-09,5348.299805,5368.200195,5312.100098,5322.950195,5322.950195,0.0,-0.009333,0.086433,0.203622,-0.009333,-0.009333
3,3,2012-08-10,5308.200195,5330.100098,5294.100098,5320.399902,5320.399902,0.0,0.062186,0.026426,0.078351,0.062186,0.062186
4,4,2012-08-13,5316.350098,5352.450195,5309.049805,5347.899902,5347.899902,0.0,0.277214,0.169700,0.110022,0.277214,0.277214
5,5,2012-08-14,5343.250000,5387.049805,5328.799805,5380.350098,5380.350098,0.0,0.331838,0.304526,0.223746,0.331838,0.331838
6,6,2012-08-16,5385.950195,5390.649902,5356.649902,5362.950195,5362.950195,0.0,0.390571,0.361204,0.333208,0.238954,0.390571
